In [1]:
import numpy as np
import math
from pprint import pprint

In [54]:
class PDF(object):
                
    def __init__(self):
        self.name = "BinnedTemplate"
        self.nps = {}
        self.samples = {}
        self.nominal = None
        self.IsDataSet = False
    def addSample(self, sample):
        self.samples[sample.name] = sample
        if self.nominal is None:
            self.nominal = sample.nominal
        else:
            self.nominal +=sample.nominal
        for nuis in sample.nps.dtype.names:
            if 'up' in nuis or 'down' in nuis:
                np_name = nuis.replace('up_','').replace('down_',"")
                self.nps[np_name] = np_name
    def setData(self, data):
        self.IsDataSet = True
        self.data = np.asarray(data)
    def pdf(self, params):
        flux = np.zeros(len(self.nominal))
        for sample in self.samples:
            sample = self.samples[sample]
            for par in params:
                print 'testing if ',par[0],'in'
                print sample.nps.dtype.names
                if 'up_' + par[0] in sample.nps.dtype.names or 'down_' + par[0] in sample.nps.dtype.names:
                    print 'adding flux'
                    flux += sample.evaluate(par[0],par[1])
        return self.nominal + flux
    
    def getIntegral(self, hist):
        return sum(hist)

In [55]:
from numpy.lib.recfunctions import append_fields

class Sample(object):
    def __init__(self, name):
        self.name = name
        self.nps = np.array([])
    def addHist(self,hist):
        assert isinstance(hist, list)
        self.nominal = np.asarray(hist)
        
    def setHistoSys(self, name, uphist, downhist, rewrite=False):
        #self.nps[name] = self.parametrize(name, len(self.nominal), uphist, downhist)
        assert len(self.nominal) == len(uphist) == len(downhist)
        if self.nps.dtype.names is None or 'nominal' not in self.nps.dtype.names:        
            self.nps = np.zeros(len(self.nominal), dtype=[
                ('nominal', np.float)
            ])
            self.nps['nominal'] = self.nominal
        if '_'.join(['up'  ,name]) in self.nps.dtype.names or '_'.join(['down'  ,name]) in self.nps.dtype.names:
            print '[WARNING] systematic already booked, use rewrite=True if you want to repalace the values'
            if rewrite :
                self.nps['_'.join(['up'  ,name])] = np.asarray(uphist)
                self.nps['_'.join(['down',name])] = np.asarray(downhist)
        else:
            self.nps = append_fields(self.nps, '_'.join(['up'  ,name]), uphist  ).data
            self.nps = append_fields(self.nps, '_'.join(['down',name]), downhist).data
    
    def PiecewiseLinear(self, alpha, I0, Iup, Idown):
        if alpha < 0:
            return (alpha*(I0-Idown))
        else:
            return (alpha*(Iup - I0))
        
    def evaluate(self, name, value):
        f = np.vectorize(self.PiecewiseLinear)
        if '_'.join(['up'  ,name]) in self.nps.dtype.names or '_'.join(['down'  ,name]) in self.nps.dtype.names:
            return f(value, self.nps['nominal'],self.nps['_'.join(['up'  ,name])],self.nps['_'.join(['down',name])])
        else:
            return np.zeros(self.nps['nominal'].shape)

## Test adding nominal to first sample

In [56]:
nominal = [10.,20.,30.,40.]
sig = Sample('signal')
sig.addHist(nominal)
print sig.nominal

[ 10.  20.  30.  40.]


## Add first systematic to sample

In [57]:
sig.setHistoSys('JES',[i*1.5 for i in nominal],[i*.6 for i in nominal], rewrite=True)
sig.setHistoSys('JEC',[i*1.1 for i in nominal],[i*.2 for i in nominal])
sig.setHistoSys('UES',[i*1.2 for i in nominal],[i*.1 for i in nominal])
pprint(sig.nps)

array([( 10.,  15.,   6.,  11.,  2.,  12.,  1.),
       ( 20.,  30.,  12.,  22.,  4.,  24.,  2.),
       ( 30.,  45.,  18.,  33.,  6.,  36.,  3.),
       ( 40.,  60.,  24.,  44.,  8.,  48.,  4.)],
      dtype=[('nominal', '<f8'), ('up_JES', '<f8'), ('down_JES', '<f8'), ('up_JEC', '<f8'), ('down_JEC', '<f8'), ('up_UES', '<f8'), ('down_UES', '<f8')])


### test evaluation

In [58]:
flux = sig.evaluate("JES",-.5)
print flux

[-2. -4. -6. -8.]


In [59]:
print sig.nominal + flux

[  8.  16.  24.  32.]


In [60]:
sig.setHistoSys('NEW',[(i*.1+1)*a for i,a in enumerate(nominal)],[(i*.1+1)*a for i,a in enumerate(nominal)])

In [61]:
params = [('JES',-.5),('JEC',.3),('NEW',1.2)]
flux = np.zeros(len(sig.nominal))
for par in params:
    flux += sig.evaluate(par[0],par[1])
print sig.nominal + flux

[  8.3  19.   32.1  47.6]


## Add another sample and form channel

In [62]:
bkgnominal = [40.,40.,40.,40.]
bkg = Sample('background')
bkg.addHist(bkgnominal)
print bkg.nominal

[ 40.  40.  40.  40.]


In [63]:
bkg.setHistoSys('JEC',[i*1.5 for i in nominal],[i*.5 for i in nominal])
print bkg.nps

[( 40.,  15.,   5.) ( 40.,  30.,  10.) ( 40.,  45.,  15.)
 ( 40.,  60.,  20.)]


In [64]:
print bkg.nps.dtype.names

('nominal', 'up_JEC', 'down_JEC')


In [65]:
pdf = PDF()
pdf.addSample(sig)
pdf.addSample(bkg)

In [66]:
print pdf.nps

{'JES': 'JES', 'NEW': 'NEW', 'UES': 'UES', 'JEC': 'JEC'}


In [68]:
params = [('JES',-.5),('JEC',-.3),('UES',1.2)]
pdf.pdf(params)

testing if  JES in
('nominal', 'up_JES', 'down_JES', 'up_JEC', 'down_JEC', 'up_UES', 'down_UES', 'up_NEW', 'down_NEW')
adding flux
testing if  JEC in
('nominal', 'up_JES', 'down_JES', 'up_JEC', 'down_JEC', 'up_UES', 'down_UES', 'up_NEW', 'down_NEW')
adding flux
testing if  UES in
('nominal', 'up_JES', 'down_JES', 'up_JEC', 'down_JEC', 'up_UES', 'down_UES', 'up_NEW', 'down_NEW')
adding flux
testing if  JES in
('nominal', 'up_JEC', 'down_JEC')
testing if  JEC in
('nominal', 'up_JEC', 'down_JEC')
adding flux
testing if  UES in
('nominal', 'up_JEC', 'down_JEC')


array([ 37.5,  47. ,  56.5,  66. ])

# Testing minimization

In [16]:
from scipy.optimize import minimize

class minimizer(object):
    def __init__(self, name, opt = 'minuit'):
        self.name = name
        if opt.lower() in ['minuit','scipy']:
            self.type = opt.lower()
        else: self.type = 'minuit'
    def NLL(self, model, params):
        return (model.data*np.log(model.pdf(params))).sum()
    
    def newNLL(self, model):
        self.model = model
        names = model.nps.keys()
        init = np.ones(len(names))
        result = minimize(self.f, init, args=names, method = 'Nelder-Mead')
        if result.success:
            return zip(names,result.x)
        else:
            raise ValueError(result.message)

    def f(self, values, names):
        model = self.model
        params = zip(names, values)
        expected = model.pdf(params)
        return -(np.sum(model.data*np.log(expected)) - np.sum(expected))

In [17]:
pdf.setData([35.,40.,45.,50.])

In [18]:
opt = minimizer('test')
print opt.NLL(pdf, params)

710.978205305


In [19]:
opt.newNLL(pdf)

[('JES', -5.2415383973634819),
 ('NEW', -1.0103629392591059e-05),
 ('UES', 5.983067102850657),
 ('JEC', 0.24999914036166015)]

# Test Channels!

In [20]:
class HistiModel:
    def __init__(self, name="ANewHistiModel"):
        self.name = name
        self.Channels = {}
        self.nps = {}
    def AddChannel(self, channel):
        if channel.IsDataSet:
            self.Channels[channel.name] = channel
        else:
            print "No Data Set - Using Asimov"
            #To Do impliment Asimov

        for norm in channel.nps:
            self.nps[norm] = channel.nps[norm]

In [21]:
amodel = HistiModel()
amodel.AddChannel(pdf)

Redefine for Models not Channels

In [22]:
class minimizer(object):
    def __init__(self, name, opt = 'minuit'):
        self.name = name
        if opt.lower() in ['minuit','scipy']:
            self.type = opt.lower()
        else: self.type = 'minuit'

    def NLL(self, model):
        self.model = model
        names = model.nps.keys()
        init = np.ones(len(names))
#        ranges = [[i*.1,i*10.] for i in init]
        result = minimize(self.f, init, args=names, method = 'BFGS')
        if result.success:
            return zip(names,result.x)
        else:
            raise ValueError(result.message)

    def f(self, values, names):
        model = self.model
        params = zip(names, values)
        nll = None
        for chan in model.Channels:
            thechannel = model.Channels[chan]
            expected = thechannel.pdf(params)
            if nll:
                nll -= (np.sum(thechannel.data*np.log(expected)) - np.sum(expected))
            else:
                nll = -(np.sum(thechannel.data*np.log(expected)) - np.sum(expected))
        return nll
        

In [23]:
opt = minimizer('test')
#print opt.NLL(amodel, params)

In [25]:
print 'result is',opt.NLL(amodel)

 result is [('JES', -2.1492861452984804), ('JEC', 0.24999796539690644), ('UES', -0.044766270312581874), ('NEW', 1.9600835642470736e-05)]
